In [ ]:
import pandas as pd
import numpy as np
import utils

from IPython.display import display, Markdown
from matplotlib import pyplot

In [ ]:
today = utils.last_day
yesterday = today - utils.one_day

display(Markdown('# Песочница по статистике по коронавирусу COVID-19'))
display(Markdown('---'))
with utils.setlocale_ctx('ru_RU'):
    display(Markdown(f"Данные в отчетах охватывают период \
         с '_{utils.first_day.date().strftime('%d %B, %Y')}_' \
         по '_{utils.last_day.date().strftime('%d %B, %Y')}_'. \
        За \"Сегодня\" принят последний доступный день в отчетах"))

In [ ]:
display(Markdown("## Топ десять стран по количеству заболевших на \"Сегодня\""))
display(Markdown("В таблице представлены ТОП-10 стран отсортированны по количеству заболевших на \"Сегодня\", а также с данными по количеству заболевших за последние 7 дней"))

utils.storage.get_countries_report_by_column("Confirmed_Change", start_date=today - utils.one_day * 6).fillna(0).sort_values(by=today, axis = 1).iloc[:,-10:]

In [ ]:
display(Markdown("## Динамика ТОП-5 стран по количеству заболевших на \"Сегодня\""))
display(Markdown("Здесь и ниже суффикс SMA означает среднее количество за N дней."))

countries_list = list(utils.storage.get_countries_report_by_column("Confirmed_Change", start_date=today).fillna(0).sort_values(by=today, axis = 1).iloc[:,-5:].columns)

pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Confirmed_Change'

for country in countries_list:
    country_df = utils.storage.get_country_report(country)
    utils.plot.bar_with_sma_line(ax, country_df[column_name], 10, country, 0.1)
ax.legend(loc='upper left')
ax.set_xlim(utils.str_to_datetime('15-03-2020'), today)
ax.set_ylim(bottom=100)
pyplot.yscale("log")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

all_countries = utils.storage.get_countries_report_by_column(column_name).fillna(0).apply(utils.data.normalize)
mean = all_countries.mean(axis=1)
median = all_countries.quantile(.5, axis=1)

utils.plot.bar_with_sma_line(ax, mean, label="Среднее")
utils.plot.bar_with_sma_line(ax, median, label="Медианное")
ax.legend()

ax.set_xlim(utils.str_to_datetime('01-03-2020'), today)
pyplot.title("Нормализованное количество заболевших в день")
pyplot.show()


In [ ]:
column_name = "Confirmed_Change"
start_date = utils.str_to_datetime('01-04-2020')

moscow_df = utils.storage.get_region_report("Russia", "Москва")[start_date:]
regions_df = utils.storage.get_regions_report_by_column("Russia", column_name, exclude=["Москва"], start_date=start_date)


In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

regions_df['Total'] = 0
regions_df['Total'] = regions_df.sum(axis = 1)

utils.plot.bar_with_sma_line(ax, regions_df.Total, 3, 'Регионы')
utils.plot.bar_with_sma_line(ax, moscow_df[column_name], 3, 'Москва')
utils.plot.key_russian_dates(ax)

#ax.set_ylim(bottom=5000, top = 6500)
ax.legend(loc='center left')
ax.set_title('Количество заболевших в день в Москве и остальной России')
ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)
pyplot.show()

In [ ]:
display(Markdown("## Динамика ТОП-10 регионов по количеству заболевших на \"Сегодня\""))
display(Markdown("Показаны топ регионов по количетсву больных в день. Кроме Москвы"))

column_name = 'Confirmed_Change'
regions_list = list(
    utils.storage.get_regions_report_by_column(
        "Russia", column_name, exclude=['Москва'],
        start_date=today).fillna(0).sort_values(by=today,
                                                axis=1).iloc[:, -10:].columns)
regions_list.reverse()
regions_df = utils.storage.get_regions_report_by_column("Russia", column_name, include=regions_list,start_date=utils.str_to_datetime('01-04-2020'))

pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()


for region in regions_list:
    utils.plot.bar_with_sma_line(ax, regions_df[region], 10, region, 0.1)
ax.legend()
ax.set_xlim(utils.str_to_datetime('01-04-2020'), today)
pyplot.show()

In [ ]:
pyplot.figure(figsize=(24,12))

ax = pyplot.subplot(111)
ax.xaxis_date()


for region_name in ["Санкт-Петербург", "Ростовская обл.", "Краснодарский край", "Хабаровский край"]:
    region_df = utils.storage.get_region_report("Russia", region_name)[start_date:]

    utils.plot.bar_with_sma_line(ax, region_df[column_name], 7, region_name+'-'+column_name, 0.2)
utils.plot.key_russian_dates(ax)

ax.legend(loc='center left')
ax.set_xlim(right=today)
pyplot.show()

In [ ]:
regions = regions_list[:4]
regions.insert(0,"Москва")
figure = pyplot.figure(figsize=(24,24))
utils.plot.report(figure, utils.storage.get_country_report("Russia"), "Россия", start_date = start_date, draw_key_dates=True)

for region in regions:
    figure = pyplot.figure(figsize=(24,24))
    utils.plot.report(figure, utils.storage.get_region_report("Russia", region), region, start_date = start_date, draw_key_dates=True)

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

country_name = "Russia"
all_regions = utils.storage.get_regions_report_by_column(country_name ,column_name, start_date=start_date).apply(utils.data.normalize)

for region in all_regions.columns:
    ax.bar(all_regions.index, all_regions[region], alpha = 1/len(all_regions), color = "Blue")

mean = all_regions.mean(axis=1)
median = all_regions.median(axis=1)
ax.plot(mean.index,mean, color = "Red", label = "Mean")
ax.plot(median.index,median, color = "Yellow", label="Median")
utils.plot.key_russian_dates(ax)
ax.axhline(1,color='black', linestyle='dashed', alpha=1)

ax.set_ylim(bottom=0, top = 1)
ax.legend()
pyplot.title("Нормализованное количество заболевших в день")
pyplot.show()


In [ ]:
utils.storage.get_regions_report_by_column(country_name ,column_name, start_date=start_date).apply(utils.data.normalize).median(axis=1)

In [ ]:
pyplot.figure(figsize=(24,12))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Rt'

for region_name in ["Санкт-Петербург", "Ростовская обл.", "Краснодарский край", "Хабаровский край"]:
    region_df = utils.storage.get_region_report("Russia", region_name)

    utils.plot.bar_with_sma_line(ax, region_df[column_name], 7, region_name+'-'+column_name, 0.1)

ax.set_ylim(0.6, 2)
utils.plot.key_russian_dates(ax)

ax.axhline(1,color='black', linestyle='dashed', alpha=1)
ax.set_xlim(start_date, today)


ax.legend(loc='upper left')
pyplot.show()

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()
utils.plot.bar_with_sma_line(ax ,(utils.storage.get_region_report("Russia", "Москва")["Confirmed_Change"] / utils.storage.get_country_report("Russia")['Confirmed_Change'])*100)
ax.set_xlim(utils.str_to_datetime("01-04-2020"))
ax.set_title('Процентное соотношение новых случаев в Москве к России')
pyplot.show()

In [ ]:
((utils.storage.get_region_report("Russia", "Москва")["Confirmed_Change"] / utils.storage.get_country_report("Russia")['Confirmed_Change'])*100).tail(28)

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()
ax.set_xlim(start_date, today)
ax.set_ylim(0,2)

country_name = "Russia"
column_name = "Rt"
all_regions = utils.storage.get_regions_report_by_column(country_name,column_name,)#.rolling(window=3).mean()

for region in all_regions.columns:
    ax.plot(all_regions.index, all_regions[region], alpha = .05, color = "Black")

mean = all_regions.mean(axis=1)
median = all_regions.median(axis=1)
#utils.plot.key_russian_dates(ax)
ax.plot(mean.index,mean, color = "Blue", label = "Средняя")
ax.plot(median.index,median, color = "Gold", label="Медиана")
ax.axhline(1, color='Red', linestyle='dashed')




ax.legend()
pyplot.title(r"$R_t$ по регионам России")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

country_name = "Russia"
column_name = "Time_To_Resolve"
all_regions = utils.storage.get_regions_report_by_column(country_name,column_name)

for region in all_regions.columns:
    ax.plot(all_regions.index, all_regions[region], alpha = .1, color = "Blue")

median = all_regions.median(axis=1)
quantile = all_regions.quantile(.75,axis = 1)
utils.plot.key_russian_dates(ax)
ax.plot(median.index,median, color = "DarkGreen", label="Медиана")
ax.plot(quantile.index,quantile, color = "Orange", label="Квантиль 75")

ax.set_xlim(start_date, today)


ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)
ax.legend()
pyplot.title(r"$TTR$ по регионам России")
pyplot.show()